In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri May  6 16:30:19 2022

@author: 工作站1
"""

from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from sklearn.svm import SVC
import numpy as np
import random
import math
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from vmdpy import VMD
from scipy.fftpack import hilbert,fft,ifft
from math import log
import pandas as pd


df = pd.read_csv('solar2019.csv',header=0, parse_dates=[0],index_col=0)# 读取CSV文件，并设置第一列为日期，作为索引
# print(df.head(10))
x = df.iloc[0:2000,-1].values
print(x)


tau = 0.  # noise-tolerance (no strict fidelity enforcement)
DC = 0  # no DC part imposed
init = 1  # initialize omegas uniformly
tol = 1e-7



# 3.初始化参数
W = 0.5                                # 惯性因子
c1 = 0.2                                # 学习因子
c2 = 0.5                                # 学习因子
n_iterations = 40                       # 迭代次数
n_particles = 20                       # 种群规模
low = [3, 100]
up = [10, 3000]
var_num = 2
bound = (low,up)


# 4.设置适应度值
def fitness_function(position):
    
    K = int(position[0])
    alpha = position[1]
    if K < bound[0][0]:
        K = bound[0][0]
    if K > bound[1][0]:
        K = bound[1][0]
        
    if alpha < bound[0][1]:
        alpha = bound[0][1]
    if alpha > bound[1][1]:
        alpha =bound[1][1]

    
    u, u_hat, omega = VMD(x, alpha, tau, K, DC, init, tol)
    #  
    EP = []
    for i in range(K):
        H = np.abs(hilbert(u[i,:]))
        e1 = []
        for j in range(len(H)):
            p = H[j]/np.sum(H)
            e = -p*log(p,2)
            e1.append(e)
        E = np.sum(e1)  
        EP.append(E)
    s = np.sum(EP)/K
    return s



## 5.粒子图
# def plot(position):
#     x = []
#     y = []
#     for i in range(0, len(particle_position_vector)):
#         x.append(particle_position_vector[i][0])
#         y.append(particle_position_vector[i][1])
#     colors = (0, 0, 0)
#     plt.scatter(x, y, c=colors, alpha=0.1)
#     # 设置横纵坐标的名称以及对应字体格式
#     #font2 = {'family': 'Times New Roman','weight': 'normal', 'size': 20,}
#     plt.xlabel('gamma')
#     plt.ylabel('C')
#     plt.axis([0, 10, 0, 10],)
#     plt.gca().set_aspect('equal', adjustable='box')
#     return plt.show()

# 6.初始化粒子位置，进行迭代
pop_x = np.zeros((n_particles,var_num))
g_best = np.zeros(var_num)
temp = -1
for i in range(n_particles):
    for j in range(var_num):
        pop_x[i][j] = np.random.rand()*(bound[1][j]-bound[0][j])+bound[0][j]
    fit = fitness_function(pop_x[i])
   
    if fit > temp:
        g_best = pop_x[i]
        temp = fit
# particle_position_vector = np.array([np.array([random.random() * 100, random.random() * 100]) for _ in range(n_particles)])
# print('zzz',particle_position_vector)
pbest_position = pop_x
pbest_fitness_value = np.zeros(n_particles)
# print(pbest_fitness_value)
gbest_fitness_value = np.zeros(var_num)
# print(gbest_fitness_value[1])
gbest_position = g_best
velocity_vector = ([np.array([0, 0]) for _ in range(n_particles)])
iteration = 0

while iteration < n_iterations:
    # plot(particle_position_vector)
    for i in range(n_particles):
        # print(pop_x[i])
        fitness_cadidate = fitness_function(pop_x[i])
        print("error of particle-", i, "is (training, test)", fitness_cadidate)
        print(" At (K, alpha): ",int(pop_x[i][0]),pop_x[i][1])

        if (pbest_fitness_value[i] > fitness_cadidate):
            pbest_fitness_value[i] = fitness_cadidate
            pbest_position[i] = pop_x[i]

        elif (gbest_fitness_value[1] > fitness_cadidate):
            gbest_fitness_value[1] = fitness_cadidate
            gbest_position = pop_x[i]

        elif (gbest_fitness_value[0] < fitness_cadidate):
            gbest_fitness_value[0] = fitness_cadidate
            gbest_position = pop_x[i]

    for i in range(n_particles):
        new_velocity = (W * velocity_vector[i]) + (c1 * random.random()) * (
                    pbest_position[i] - pop_x[i]) + (c2 * random.random()) * (
                                   gbest_position - pop_x[i])
        new_position = new_velocity + pop_x[i]
        pop_x[i] = new_position

    iteration = iteration + 1

plt.plot()
# 7.输出最终结果
print("The best position is ", int(gbest_position[0]),gbest_position[1], "in iteration number", iteration, "with error (train, test):",
      fitness_function(gbest_position))


[0. 0. 0. ... 0. 0. 0.]
error of particle- 0 is (training, test) 10.62178048674916
 At (K, alpha):  3 2091.153638078027
error of particle- 1 is (training, test) 10.154185443507039
 At (K, alpha):  9 2321.8784167001113
error of particle- 2 is (training, test) 10.166767521338086
 At (K, alpha):  9 1486.9813386191222
error of particle- 3 is (training, test) 10.334358489390231
 At (K, alpha):  3 343.5902680008072
error of particle- 4 is (training, test) 9.99852555386683
 At (K, alpha):  8 243.41884797561318
error of particle- 5 is (training, test) 10.275736556475204
 At (K, alpha):  5 590.9517557726446
error of particle- 6 is (training, test) 10.166877353115584
 At (K, alpha):  8 2513.697221786922
error of particle- 7 is (training, test) 10.620279165819989
 At (K, alpha):  3 1732.6410239202746
error of particle- 8 is (training, test) 10.089431178612243
 At (K, alpha):  7 726.0833038884002
error of particle- 9 is (training, test) 10.191558227961377
 At (K, alpha):  7 2102.5660330607943
erro

error of particle- 4 is (training, test) 10.27758161324915
 At (K, alpha):  4 2119.005787675743
error of particle- 5 is (training, test) 10.279956482209084
 At (K, alpha):  4 2194.001711520077
error of particle- 6 is (training, test) 10.2976525264319
 At (K, alpha):  4 2655.5862728751395
error of particle- 7 is (training, test) 10.62318263242578
 At (K, alpha):  3 2430.4485471206112
error of particle- 8 is (training, test) 10.27862720185763
 At (K, alpha):  4 2017.260132921027
error of particle- 9 is (training, test) 10.299351739500063
 At (K, alpha):  5 2391.5217800848613
error of particle- 10 is (training, test) 10.621031093599518
 At (K, alpha):  3 1906.570050413611
error of particle- 11 is (training, test) 10.280017504735582
 At (K, alpha):  4 2196.1547682831997
error of particle- 12 is (training, test) 10.283003723368484
 At (K, alpha):  4 2283.1528406885327
error of particle- 13 is (training, test) 10.623049021910282
 At (K, alpha):  3 2396.051348190866
error of particle- 14 is (

error of particle- 8 is (training, test) 10.289738503097373
 At (K, alpha):  4 2464.4831530455863
error of particle- 9 is (training, test) 10.29229069238172
 At (K, alpha):  4 2529.1862472734547
error of particle- 10 is (training, test) 10.623563840462038
 At (K, alpha):  3 2527.0455143587237
error of particle- 11 is (training, test) 10.623666184660118
 At (K, alpha):  3 2552.701036027723
error of particle- 12 is (training, test) 10.291327856996482
 At (K, alpha):  4 2507.356824812347
error of particle- 13 is (training, test) 10.623929070596157
 At (K, alpha):  3 2618.429702220243
error of particle- 14 is (training, test) 10.621324512424396
 At (K, alpha):  3 1978.4636857671458
error of particle- 15 is (training, test) 10.624148143527496
 At (K, alpha):  3 2676.833251343874
error of particle- 16 is (training, test) 10.292313849086806
 At (K, alpha):  4 2529.749559175358
error of particle- 17 is (training, test) 10.623847198413996
 At (K, alpha):  3 2597.6041967698843
error of particle-

error of particle- 13 is (training, test) 10.624067680061428
 At (K, alpha):  3 2654.895936391112
error of particle- 14 is (training, test) 10.623469647885141
 At (K, alpha):  3 2504.430647535326
error of particle- 15 is (training, test) 10.624154626794072
 At (K, alpha):  3 2678.626304721936
error of particle- 16 is (training, test) 10.623983049334884
 At (K, alpha):  3 2632.435623090332
error of particle- 17 is (training, test) 10.624083024746808
 At (K, alpha):  3 2659.0349143636968
error of particle- 18 is (training, test) 10.62415771580433
 At (K, alpha):  3 2679.4680464963544
error of particle- 19 is (training, test) 10.624098748657866
 At (K, alpha):  3 2663.297769215726
error of particle- 0 is (training, test) 10.62410567075627
 At (K, alpha):  3 2665.181367822652
error of particle- 1 is (training, test) 10.624136490357609
 At (K, alpha):  3 2673.6200581828934
error of particle- 2 is (training, test) 10.624009133323725
 At (K, alpha):  3 2639.292427173967
error of particle- 3 i

In [3]:
import numpy as np
import random
from scipy.fftpack import hilbert
from math import log
from vmdpy import VMD
import pandas as pd

df = pd.read_csv('solar2019.csv', header=0, parse_dates=[0], index_col=0)
x = df.iloc[0:2000, -1].values

# PSO 参数
W = 0.5  # 惯性因子
c1 = 0.2  # 个体学习因子
c2 = 0.5  # 群体学习因子
n_iterations = 40  # 迭代次数
n_particles = 20  # 粒子数量
var_num = 2
low = [3, 100]
up = [10, 3000]
bound = (low, up)

# 适应度函数
def fitness_function(position):
    K = int(position[0])
    alpha = position[1]
    if K < bound[0][0]:
        K = bound[0][0]
    if K > bound[1][0]:
        K = bound[1][0]

    if alpha < bound[0][1]:
        alpha = bound[0][1]
    if alpha > bound[1][1]:
        alpha = bound[1][1]

    u, _, _ = VMD(x, alpha, 0, K, 0, 1, 1e-7)

    EP = []
    for i in range(K):
        H = np.abs(hilbert(u[i, :]))
        e1 = [-p * log(p, 2) for p in H / np.sum(H)]
        E = np.sum(e1)
        EP.append(E)

    s = np.sum(EP) / K
    return s

# 初始化粒子位置和速度
pop_x = np.random.uniform(low=bound[0], high=bound[1], size=(n_particles, var_num))
pbest_position = np.copy(pop_x)
pbest_fitness_value = np.zeros(n_particles)
gbest_fitness_value = np.inf
gbest_position = np.zeros(var_num)
velocity_vector = np.zeros((n_particles, var_num))

# 迭代
iteration = 0
while iteration < n_iterations:
    for i in range(n_particles):
        fitness_cadidate = fitness_function(pop_x[i])
        print("Error of particle-", i, "is:", fitness_cadidate)
        print("At (K, alpha):", int(pop_x[i][0]), pop_x[i][1])

        if pbest_fitness_value[i] > fitness_cadidate:
            pbest_fitness_value[i] = fitness_cadidate
            pbest_position[i] = np.copy(pop_x[i])

        if gbest_fitness_value > fitness_cadidate:
            gbest_fitness_value = fitness_cadidate
            gbest_position = np.copy(pop_x[i])

    for i in range(n_particles):
        new_velocity = (W * velocity_vector[i]) + (c1 * random.random()) * (pbest_position[i] - pop_x[i]) + (
                    c2 * random.random()) * (gbest_position - pop_x[i])
        new_position = new_velocity + pop_x[i]
        pop_x[i] = new_position

    iteration += 1

# 输出最终结果
print("The best position is ", int(gbest_position[0]), gbest_position[1], "in iteration number", iteration,
      "with error:", gbest_fitness_value)

Error of particle- 0 is: 10.084984947509309
At (K, alpha): 8 1325.3317458112365
Error of particle- 1 is: 9.97777619306402
At (K, alpha): 9 351.20461091176855
Error of particle- 2 is: 10.049002704288908
At (K, alpha): 7 243.61839010971843
Error of particle- 3 is: 10.025924593405998
At (K, alpha): 6 274.4054136674779
Error of particle- 4 is: 10.266081593678253
At (K, alpha): 5 1613.56875590773
Error of particle- 5 is: 10.229288911968434
At (K, alpha): 7 2797.470788388848
Error of particle- 6 is: 10.491111960313505
At (K, alpha): 4 544.7221245936214
Error of particle- 7 is: 10.119832339395408
At (K, alpha): 9 835.1069187103483
Error of particle- 8 is: 10.030795866674872
At (K, alpha): 5 220.93010689414268
Error of particle- 9 is: 10.172765905206221
At (K, alpha): 7 1801.1567211101703
Error of particle- 10 is: 10.62148618953266
At (K, alpha): 3 2016.4710652825472
Error of particle- 11 is: 10.306926600107824
At (K, alpha): 6 2987.453837639263
Error of particle- 12 is: 10.133148128085734
At 

Error of particle- 4 is: 10.033551788019293
At (K, alpha): 8 743.2812505889353
Error of particle- 5 is: 10.115515857521705
At (K, alpha): 8 1712.1025426390054
Error of particle- 6 is: 10.005044588090513
At (K, alpha): 8 398.9699759607155
Error of particle- 7 is: 10.097947244973865
At (K, alpha): 9 554.875780664056
Error of particle- 8 is: 10.000825884652759
At (K, alpha): 8 313.74614689276285
Error of particle- 9 is: 10.121751239692763
At (K, alpha): 9 858.6711787489855
Error of particle- 10 is: 10.113006817242551
At (K, alpha): 7 1000.9636706642854
Error of particle- 11 is: 10.099889595716983
At (K, alpha): 8 1508.5807024503795
Error of particle- 12 is: 10.103322055243183
At (K, alpha): 9 619.8150045243316
Error of particle- 13 is: 10.129892386366569
At (K, alpha): 7 1205.8533363042066
Error of particle- 14 is: 10.07514994663265
At (K, alpha): 9 495.4082951444656
Error of particle- 15 is: 10.093275496021533
At (K, alpha): 7 770.0300961070596
Error of particle- 16 is: 10.06862958219288

Error of particle- 8 is: 10.000384448717488
At (K, alpha): 8 302.91977023278554
Error of particle- 9 is: 10.04704619672989
At (K, alpha): 8 886.5501050524309
Error of particle- 10 is: 10.2898608452212
At (K, alpha): 6 1033.121227838633
Error of particle- 11 is: 10.164754870659356
At (K, alpha): 7 1682.5466122928406
Error of particle- 12 is: 10.069838011069455
At (K, alpha): 9 477.7082156037186
Error of particle- 13 is: 10.140206040736889
At (K, alpha): 7 1341.6551287316495
Error of particle- 14 is: 10.030130071007765
At (K, alpha): 8 707.9969307838859
Error of particle- 15 is: 10.085672813925516
At (K, alpha): 7 685.8176009321324
Error of particle- 16 is: 10.031919296098145
At (K, alpha): 8 726.2234438586826
Error of particle- 17 is: 10.101548747817466
At (K, alpha): 9 595.5713312304948
Error of particle- 18 is: 10.054143073553526
At (K, alpha): 8 966.2118133764548
Error of particle- 19 is: 10.06111628882535
At (K, alpha): 8 1044.980816474428
Error of particle- 0 is: 10.101693265916861

Error of particle- 12 is: 10.024508720212594
At (K, alpha): 8 646.6836727503049
Error of particle- 13 is: 10.0384795889556
At (K, alpha): 8 794.5659425359815
Error of particle- 14 is: 10.022080150180521
At (K, alpha): 8 619.7125707595167
Error of particle- 15 is: 10.07753953446509
At (K, alpha): 7 599.8742717730827
Error of particle- 16 is: 10.044623422746193
At (K, alpha): 8 859.8935047546491
Error of particle- 17 is: 10.019911656460057
At (K, alpha): 8 594.9901951426127
Error of particle- 18 is: 10.06174989939558
At (K, alpha): 8 1052.07455492793
Error of particle- 19 is: 10.050598607049915
At (K, alpha): 8 925.9212239906583
Error of particle- 0 is: 10.098772266921578
At (K, alpha): 9 562.4724674060647
Error of particle- 1 is: 9.973303620636564
At (K, alpha): 9 334.5457088949521
Error of particle- 2 is: 10.000403028427154
At (K, alpha): 8 303.3836920683665
Error of particle- 3 is: 10.000678424084702
At (K, alpha): 8 310.1049455108731
Error of particle- 4 is: 10.032084222626668
At (K,

Error of particle- 16 is: 10.152416083487974
At (K, alpha): 7 1503.0492059196333
Error of particle- 17 is: 10.01702287133776
At (K, alpha): 8 559.8795576251199
Error of particle- 18 is: 10.062463304181314
At (K, alpha): 8 1060.1287010181568
Error of particle- 19 is: 10.046946048787458
At (K, alpha): 8 885.4457235229394
Error of particle- 0 is: 10.106992573930249
At (K, alpha): 9 670.5524211871624
Error of particle- 1 is: 9.972105568820613
At (K, alpha): 9 328.5841769561768
Error of particle- 2 is: 10.000360070281129
At (K, alpha): 8 302.29211168926884
Error of particle- 3 is: 10.000470663902476
At (K, alpha): 8 305.03482508845605
Error of particle- 4 is: 10.029670495427744
At (K, alpha): 8 703.3601862762455
Error of particle- 5 is: 10.036101182000678
At (K, alpha): 8 769.3646852644177
Error of particle- 6 is: 10.059873216037088
At (K, alpha): 7 415.85210106563966
Error of particle- 7 is: 10.054062547642156
At (K, alpha): 9 428.11315842081956
Error of particle- 8 is: 10.000029314273695


Error of particle- 0 is: 10.097061522002305
At (K, alpha): 9 547.4587831088085
Error of particle- 1 is: 9.973901710199375
At (K, alpha): 9 337.39772355215183
Error of particle- 2 is: 10.000110952888512
At (K, alpha): 8 295.2980804591689
Error of particle- 3 is: 10.00048641287427
At (K, alpha): 8 305.4168160198469
Error of particle- 4 is: 10.022385103068908
At (K, alpha): 8 623.1570702081285
Error of particle- 5 is: 10.048468985731434
At (K, alpha): 8 902.49524827557
Error of particle- 6 is: 10.00325766899468
At (K, alpha): 8 370.30970083051614
Error of particle- 7 is: 10.083909289319077
At (K, alpha): 9 505.1168012921915
Error of particle- 8 is: 10.050049657639974
At (K, alpha): 7 279.3489133586802
Error of particle- 9 is: 10.029370982114013
At (K, alpha): 8 700.2503874029062
Error of particle- 10 is: 10.099475794049189
At (K, alpha): 7 842.4241439639407
Error of particle- 11 is: 10.044853161413474
At (K, alpha): 8 862.3989471472275
Error of particle- 12 is: 10.02396075985842
At (K, al

Error of particle- 4 is: 10.02719761445336
At (K, alpha): 8 676.523492455684
Error of particle- 5 is: 10.0466892080642
At (K, alpha): 8 882.626923034748
Error of particle- 6 is: 10.048011632296829
At (K, alpha): 6 421.12394926045
Error of particle- 7 is: 10.093247872739818
At (K, alpha): 9 528.4601082284812
Error of particle- 8 is: 10.00031620826006
At (K, alpha): 8 301.08094124680963
Error of particle- 9 is: 10.033791276739203
At (K, alpha): 8 745.7350518970496
Error of particle- 10 is: 10.104663717061948
At (K, alpha): 7 902.6797436054143
Error of particle- 11 is: 10.07830112350382
At (K, alpha): 8 1243.9533410587019
Error of particle- 12 is: 10.02818783608421
At (K, alpha): 8 687.5029345374005
Error of particle- 13 is: 10.04210368416546
At (K, alpha): 8 832.9702176261133
Error of particle- 14 is: 10.018280405289213
At (K, alpha): 8 575.0989545089542
Error of particle- 15 is: 10.014592067021102
At (K, alpha): 8 529.3787101962814
Error of particle- 16 is: 10.122662916879715
At (K, alp

Error of particle- 8 is: 10.000492291144912
At (K, alpha): 8 305.5601476653687
Error of particle- 9 is: 10.025138672527536
At (K, alpha): 8 653.6027773773446
Error of particle- 10 is: 10.08511199288244
At (K, alpha): 7 679.9391242486222
Error of particle- 11 is: 10.083687106441609
At (K, alpha): 8 1309.274188935565
Error of particle- 12 is: 10.02394522464774
At (K, alpha): 8 640.500412516404
Error of particle- 13 is: 10.122070411415942
At (K, alpha): 7 1109.738635249534
Error of particle- 14 is: 10.032160248898323
At (K, alpha): 8 728.7350539729483
Error of particle- 15 is: 10.017764342796367
At (K, alpha): 8 568.8064518666795
Error of particle- 16 is: 10.057414303911626
At (K, alpha): 8 1003.1080307133145
Error of particle- 17 is: 10.01819045110122
At (K, alpha): 8 573.994842056375
Error of particle- 18 is: 10.069181023280894
At (K, alpha): 8 1136.175125883685
Error of particle- 19 is: 10.071798827722928
At (K, alpha): 8 1166.5370581375532
Error of particle- 0 is: 10.10720769869368
At